#Las matemáticas tras la regresión logística

##Las tablas de contingencia

In [ ]:
import pandas as pd

In [ ]:
url = 'https://raw.githubusercontent.com/javilledo/python-ml-course/master/datasets/gender-purchase/Gender%20Purchase.csv'
data = pd.read_csv(url)

In [ ]:
data.head()

,Gender,Purchase
0,Female,Yes
1,Female,Yes
2,Female,No
3,Male,No
4,Male,Yes


In [ ]:
data.shape

(511, 2)

In [ ]:
contingency_table = pd.crosstab(data['Gender'], data['Purchase'])
contingency_table

Purchase,No,Yes
Gender,,
Female,106,159
Male,125,121


In [ ]:
contingency_table.sum(axis=0)

Purchase
No     231
Yes    280
dtype: int64

In [ ]:
contingency_table.sum(axis=1)

Gender
Female    265
Male      246
dtype: int64

In [ ]:
contingency_table.astype('float').div(contingency_table.sum(axis=1),axis=0) #De cada género, saber su distribución por cada respuesta
#PROBABILIDADES CON EL GÉNERO CONDICIONADO

Purchase,No,Yes
Gender,,
Female,0.40000,0.60000
Male,0.50813,0.49187


In [ ]:
contingency_table.astype('float').div(contingency_table.sum(axis=0),axis=1) #De cada respuesta, saber su distribución por género
#PROBABILIDADES CON LA COMPRA CONDICIONADA

Purchase,No,Yes
Gender,,
Female,0.458874,0.567857
Male,0.541126,0.432143


##La probabilidad condicional

In [ ]:
from IPython.display import display, Math, Latex

In [ ]:
display(Math(r'x^2')) # para escribir la fórmula en Latex, cuidado con poner la r, pero también se puede en la parte de texto

<IPython.core.display.Math object>

* La probabilidad de que un cliente compre un producto siendo un hombre.
$P(Purchase|Male)=\frac{Número\ total\ de\ compras\ hechas\ por\ hombres}{Número\ de\ hombres} = \frac{Purchase\cap Male}{Male}$


La probabilidad de que una persona que compra un producto sea una mujer. $P(Female|Purchase)=\frac{Número\ total\ de\ compras\ hechas\ por\ mujeres}{Número\ de\ compras} = \frac{Purchase\cap Female}{Purchase}$


Ojo que es diferente la afirmación, los valores no son complementarios.

In [ ]:
121/(246)

0.491869918699187

In [ ]:
159/280

0.5678571428571428

##RATIO DE PROBABILIDADES
Cociente entre los casos de éxito entre los de fracaso en el suceso estudiado para cada grupo

$P_m = Probabilidad\ de\ hacer\ compra\ sabiendo\ que\ es\ hombre$

$P_f = Probabilidad\ de\ hacer\ compra\ sabiendo\ que\ es\ mujer$

$odds_{purchase, male} = \frac{P_m}{1-Pm}$

$odds_{purchase, female} = \frac{P_f}{1-Pf}$

$odds \in [0,+\infty]$

In [ ]:
pm = 121 / 246
pf = 159 / 265
odds_m=pm/(1-pm) #es equivalente a los hombres que compran respecto a los que no compran, 121/125
odds_f = pf/(1-pf) #es equivalente a las mujeres que compran respecto a las que no compran, 159/106
pm,pf,odds_m, odds_f

(0.491869918699187, 0.6, 0.9680000000000002, 1.4999999999999998)

* Si el ratio odds es superior a 1, es más probable el éxito que el fracaso
* Si el ratio odds es exactamente 1, es equiprobable
* Si el ratio odds es inferior a 1, es más probable el fracaso que el éxito

##La regresión logística desde la regresión lineal
En el caso de la regresión lineal: $ y = \alpha + \beta \cdot x\ con (x,y)\in[-\infty,+\infty]^2$

Pero si lo que queremos es categorizar, o hacer una predicción sobre una variable categórica, la variable a predecir ya no podrá ser continua, sino que tendrá que ser discreta: $y \in \{0,1\}$, pero realmente lo que se preciden no es 0 ó 1, sino una probabilidad de cada uno de ellos: $P \in [0,1]$ con $P = \alpha + \beta \cdot X$

P es la probabilidad condicionada a la presencia de la variable X

Pero lo que ocurre es que como $X\in[-\infty,+\infty]$, también $X\in[-\infty,+\infty]$, cuestión que no puede ocurrir porque al ser una probabilidad debería estar entre 0 y 1. Por ello, lo que se empleará será odds, de forma que $\frac{P}{1-P} \in [0,+\infty]$ con $\frac{P}{1-P} = \alpha + \beta \cdot X$, que al menos ya es un paso.

Para conseguir limitar por la derecha el $\infty$ a 1, se usará el logarítmo neperiano:

$\ln(\frac{P}{1-P}) = \alpha + \beta \cdot X$

De esa forma,

$\frac{P}{1-P}\in[0,1]\rightarrow ln(\frac{P}{1-P})\in[-\infty,0]$

$\frac{P}{1-P}\in[1,+\infty]\rightarrow ln(\frac{P}{1-P})\in[0,+\infty]$

Así, pasando el logaritmo neperiano como la exponencial al otro lado:

$\frac{P}{1-P} = e^{\alpha + \beta \cdot X}$

$P = (1-P)\cdot e^{\alpha + \beta \cdot X}$

$P = e^{\alpha + \beta \cdot X} - P \cdot e^{\alpha + \beta \cdot X}$

$P + P \cdot e^{\alpha + \beta \cdot X}= e^{\alpha + \beta \cdot X}$

$P \cdot (1+e^{\alpha + \beta \cdot X})= e^{\alpha + \beta \cdot X}$

$P = \frac{e^{\alpha + \beta \cdot X}}{1+e^{\alpha + \beta \cdot X}}=\frac{1}{1+e^{-(\alpha + \beta \cdot X)}}\in[0,1]$

Y como resumen de casos de la fórmula anterior, puede obtenerse lo siguiente:

* Si $\alpha + \beta \cdot X \rightarrow -\infty \Rightarrow P \rightarrow 0$

* Si $\alpha + \beta \cdot X = 0 \Rightarrow P = 0.5$

* Si $\alpha + \beta \cdot X \rightarrow \infty \Rightarrow P \rightarrow 1$

##Regresión logística múltiple

Es tan sencillo como hacer la equivalencia 

$P = \frac{e^{\alpha + \beta \cdot X}}{1+e^{\alpha + \beta \cdot X}}=\frac{1}{1+e^{-(\alpha + \sum_{i=1}^n(\beta_i \cdot x_i))}}\in[0,1]$